In [ ]:
import string

class SearcherIIndexVII(SearcherIIndex):
    """倒排索引文本搜索实现类(改进)
    
    自定义解析，保留英文片段，将中文片段多粒度分词处理
    
    Attributes:
        index: 检索使用的倒排索引
        max_id: 当前索引的文档最大ID
        doc_list: 索引文档原文
    """
    def parse_doc(self, doc):
        """对文档进行自定义解析，保留英文串，对中文串多粒度分词
        
        Args:
            doc:待解析的原始文档
        
        Returns:
            解析结果列表，元素是切分得到的term
        """
        result = []
        state_last = ''
        cache = ''
        for c in doc:
            state_c = c in string.ascii_letters \
                or c.isdigit() \
                or c in ('-', ':', '.')
            if c == ' ':
                if state_last:
                    result.append(cache)
                else:
                    result.extend(list(jieba.cut_for_search(cache)))
                result.append(' ')
                cache = ''
                state_last = '' 
            else:
                if state_c == state_last:
                    cache += c
                else:
                    if state_last != '':
                        result.append(cache)
                    else:
                        result.extend(list(jieba.cut_for_search(cache)))
                    cache = c
                state_last = state_c
        if cache:
            if state_last:
                result.append(cache)
            else:
                result.extend(list(jieba.cut_for_search(cache)))
        return result
    
    def add_doc(self, doc):
        """向索引中添加新文档
        
        Args:
            doc:待检索的文档(文本)
        
        Returns:
            新增文档ID
        """
        self.doc_list.append(doc)
        for term in self.parse_doc(doc):
            #构建和更新各Term对应的Posting(集合)
            if term in self.index: 
                self.index[term].add(self.max_id)
            else:
                self.index[term] = set([self.max_id])
        self.max_id += 1
        return self.max_id - 1
    
    def dumpIndex(self):
        """原样输出索引，用于检查索引构建结果
        
        Returns:
            对索引(字典结构)的Dump输出
        """
        print(self.index)
    
    def parse_query(self, doc):
        """对查询进行自定义解析，保留英文串，对中文串分词处理
        
        Args:
            doc:待解析的原始文档
        
        Returns:
            解析结果列表，元素是带有串类型标记(首字符，e为英文，c为中文)
            的切分term结果
        """
        doc = doc + ' '
        result = []
        state_last = ''
        cache = ''
        for c in doc:
            state_c = c in string.ascii_letters \
                or c.isdigit() \
                or c in ('-', ':', '.')
            flag = None
            #增加串标记，e为英文，c为中文(未来可能扩充)
            if state_c:
                flag = 'e'
            else:
                flag = 'c'
                
            if c == ' ':
                if state_last != '':
                    result.append(cache)
                    result.append('s ')
                    cache = ''
                    state_last = '' 
            elif c == '(' :
                result.append('s' + c)
                if cache != '':
                    result.append(cache)
                    cache = ''
                state_last = ''
            elif  c == ')':
                if cache != '':
                    result.append(cache)
                    result.append('s' + c)
                    cache = ''
                state_last = ''
            else:
                if state_c == state_last:
                    cache += c
                else:
                    if state_last != '':
                        result.append(cache)
                    cache = flag + c
                state_last = state_c
                
        return result
    
    def conv_query(self, query):
        """将用户的查询转换成用eval可运行、返回结果ID集合的代码段
        
        Args:
            query:待转换的原始查询字符串
        
        Returns:
            转换完成可通过eval执行返回ID集合的代码段字符串
        """
        query_new_parts = []
        all_parts = list(self.parse_query(query))
        idx = 0
        cache = '' #缓存变量，用于回收分词过程被切开的短语片段
        count_parts = len(all_parts)
        while idx < count_parts:
            if all_parts[idx][1:] == '(' or all_parts[idx][1:] == ')':
                query_new_parts.append(all_parts[idx][1:])
            elif all_parts[idx][1:] == ' ':
                query_new_parts.append(' ')
            elif all_parts[idx][1:] in ('and', 'AND', '+'):
                query_new_parts.append('&')
            elif all_parts[idx][1:] in ('or', 'OR'):
                query_new_parts.append('|')
            elif all_parts[idx][1:] in ('not', 'NOT', '-'):
                query_new_parts.append('-')
            elif (idx + 1 < count_parts #被分词切开的短语部分回收至缓存
                  and all_parts[idx+1][1:] not in (' ', ')')): 
                cache += all_parts[idx][1:]
            elif (idx + 2 < count_parts #处理词间空格的形式
                  and all_parts[idx+1][1:] == " " 
                  and all_parts[idx+2][1:] not in ('(', ')', 'and', 'AND', '+', 'or', 'OR', 'NOT', 'not', '+', '-', ' ')): 
                query_new_parts.append("{} & ".format(self.conv_part(all_parts[idx])))
                idx += 2
                continue
            else:
                query_new_parts.append(self.conv_part(cache + all_parts[idx]))
                cache = '' #合并完成清空缓存
            idx += 1
        query_new = ''.join(query_new_parts)
        return query_new
    
    def term_match(self, term):
        return self.index.get(term, set()) 
    
    def conv_part(self, part):
        flag = part[0]
        if flag == 'e':
            return "self.term_match('{}')".format(part[1:])
        elif flag == 'c':
            return "(self.term_match('{}'))".format(
                "') & self.term_match('".join(jieba.cut(part[1:])))

In [ ]:
searcher = SearcherIIndexVII('titles.txt')

query = '3-0 (中国美国)'
print(searcher.parse_query(query))
print(searcher.conv_query(query))
result = searcher.search(query)
if result:
    for doc in result:
        display(HTML(doc))
else:
    print('No result.')